## Deribit API Request  - For BTC and ETH Options Data
#### ALL Options data 

### Specfic Settlement Period ( Day, Week, Month) 

In [1]:
import pandas as pd
import json
import requests
import re
from datetime import datetime
from tqdm import tqdm
import concurrent.futures

# Functions
def get_option_name_and_settlement(coin):
    """
    :param coin: crypto-currency coin name ('BTC', 'ETH')
    :return: 2 lists:
                        1. list of traded options for the selected coin;
                        2. list of settlement period for the selected coin.
    """
    r = requests.get("https://test.deribit.com/api/v2/public/get_instruments?currency=" + coin + "&kind=option")
    result = json.loads(r.text)
    # Get option name
    name = pd.json_normalize(result['result'])['instrument_name']
    name = list(name)

    # Get option settlement period
    settlement_period = pd.json_normalize(result['result'])['settlement_period']
    settlement_period = list(settlement_period)

    return name, settlement_period

def fetch_option_data(option_name):
    """Fetch the option data for a given option name."""
    r = requests.get(f'https://test.deribit.com/api/v2/public/get_order_book?instrument_name={option_name}')
    result = json.loads(r.text)
    return pd.json_normalize(result['result'])


def extract_details(instrument_name, coin):
    """
    Extract expiration date, strike price, and option type (call or put) from instrument name.
    Adjusts to include selected coin (e.g., 'BTC' or 'ETH').
    """
    match = re.match(fr"{coin}-(\d+[A-Z]{{3}}\d+)-(\d+)-([CP])", instrument_name)
    if match:
        expiration_date = match.group(1)
        strike_price = match.group(2)
        option_type = 'Call' if match.group(3) == 'C' else 'Put'
        return expiration_date, strike_price, option_type
    return None, None, None



def get_option_data(coin, settlement_per):
    """
    :param coin: crypto-currency coin name ('BTC', 'ETH')
    :return: pandas DataFrame with all option data for a given coin, filtered for options with specified settlement.
    """
    # Get option name and settlement
    coin_name, settlement_period = get_option_name_and_settlement(coin)

    # Filter options that have the specified settlement period
    coin_name_filtered = [coin_name[i] for i in range(len(coin_name)) if settlement_period[i] == settlement_per]

    # Initialize progress bar
    pbar = tqdm(total=len(coin_name_filtered))

    # Fetch data concurrently using ThreadPoolExecutor
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_option = {executor.submit(fetch_option_data, name): name for name in coin_name_filtered}
        coin_df = []
        
        for future in concurrent.futures.as_completed(future_to_option):
            try:
                data = future.result()
                data['settlement_period'] = settlement_per
                coin_df.append(data)
            except Exception as exc:
                print(f'Error fetching data: {exc}')
            pbar.update(1)

    # Finalize DataFrame
    if len(coin_df) > 0:
        coin_df = pd.concat(coin_df)

    # Remove unnecessary columns
    columns = ['state', 'estimated_delivery_price']
    if not coin_df.empty:
        coin_df.drop(columns, inplace=True, axis=1)

    #coin_df['Expiration Date'], coin_df['Strike Price'] = zip(*coin_df['instrument_name'].apply(lambda x: extract_details(x, coin)))

    coin_df['Expiration Date'], coin_df['Strike Price'], coin_df['Option Type'] = zip(*coin_df['instrument_name'].apply(lambda x: extract_details(x, coin)))
    
    today = datetime.today()
    coin_df['Time to Expiration'] = coin_df['Expiration Date'].apply(lambda x: (datetime.strptime(x, '%d%b%y') - today).days / 365 if x else None)

    coin_df.to_csv('data/data.csv', index=False)
 
    pbar.close()

    return coin_df


In [2]:
data = get_option_data('BTC','month')
data=data[["instrument_name", "Option Type", 
           'mark_price', 'underlying_price', 'mark_iv', 
           'greeks.vega', 'Expiration Date', 'Strike Price', 'Time to Expiration']]

data['Strike Price'] = pd.to_numeric(data['Strike Price'], errors='coerce').astype('float64')

 24%|██▎       | 149/634 [00:02<00:07, 64.14it/s]

Error fetching data: 'result'
Error fetching data: 'result'
Error fetching data: 'result'
Error fetching data: 'result'


 26%|██▋       | 168/634 [00:02<00:06, 71.47it/s]

Error fetching data: 'result'
Error fetching data: 'result'
Error fetching data: 'result'
Error fetching data: 'result'
Error fetching data: 'result'
Error fetching data: 'result'
Error fetching data: 'result'
Error fetching data: 'result'
Error fetching data: 'result'
Error fetching data: 'result'
Error fetching data: 'result'
Error fetching data: 'result'
Error fetching data: 'result'


 30%|██▉       | 189/634 [00:03<00:07, 62.73it/s]

...

 96%|█████████▌| 609/634 [00:09<00:00, 67.63it/s]

Error fetching data: 'result'
Error fetching data: 'result'
Error fetching data: 'result'
Error fetching data: 'result'
Error fetching data: 'result'
Error fetching data: 'result'
Error fetching data: 'result'
Error fetching data: 'result'
Error fetching data: 'result'
Error fetching data: 'result'
Error fetching data: 'result'
Error fetching data: 'result'
Error fetching data: 'result'
Error fetching data: 'result'


 99%|█████████▉| 630/634 [00:10<00:00, 70.95it/s]C:\Users\ethan\AppData\Local\Temp\ipykernel_15132\1889503572.py:81: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  coin_df = pd.concat(coin_df)
100%|██████████| 634/634 [00:10<00:00, 62.66it/s]

Error fetching data: 'result'
Error fetching data: 'result'
Error fetching data: 'result'
Error fetching data: 'result'
Error fetching data: 'result'
Error fetching data: 'result'
Error fetching data: 'result'
Error fetching data: 'result'
Error fetching data: 'result'
Error fetching data: 'result'



C:\Users\ethan\AppData\Local\Temp\ipykernel_15132\247996183.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Strike Price'] = pd.to_numeric(data['Strike Price'], errors='coerce').astype('float64')


In [3]:
data.isnull().sum()

instrument_name       0
Option Type           0
mark_price            0
underlying_price      0
mark_iv               0
greeks.vega           0
Expiration Date       0
Strike Price          0
Time to Expiration    0
dtype: int64

In [4]:
data

,instrument_name,Option Type,mark_price,underlying_price,mark_iv,greeks.vega,Expiration Date,Strike Price,Time to Expiration
0,BTC-29NOV24-34000-P,Put,0.0001,72136.16,92.89,1.01404,29NOV24,34000.0,0.079452
0,BTC-29NOV24-32000-P,Put,0.0001,72136.16,93.16,0.51601,29NOV24,32000.0,0.079452
0,BTC-29NOV24-38000-C,Call,0.4737,72136.16,92.89,3.21594,29NOV24,38000.0,0.079452
0,BTC-29NOV24-36000-C,Call,0.5012,72136.16,92.89,1.87570,29NOV24,36000.0,0.079452
0,BTC-29NOV24-40000-P,Put,0.0008,72136.16,92.32,4.80946,29NOV24,40000.0,0.079452
...,...,...,...,...,...,...,...,...,...
0,BTC-26SEP25-120000-P,Put,0.6989,73786.36,58.33,234.12147,26SEP25,120000.0,0.904110
0,BTC-26SEP25-140000-C,Call,0.0492,73787.48,59.61,195.88070,26SEP25,140000.0,0.904110
0,BTC-26SEP25-140000-P,Put,0.9466,73787.48,59.60,195.87219,26SEP25,140000.0,0.904110
0,BTC-26SEP25-160000-C,Call,0.0376,73787.48,62.09,167.37643,26SEP25,160000.0,0.904110


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 294 entries, 0 to 0
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   instrument_name     294 non-null    object 
 1   Option Type         294 non-null    object 
 2   mark_price          294 non-null    float64
 3   underlying_price    294 non-null    float64
 4   mark_iv             294 non-null    float64
 5   greeks.vega         294 non-null    float64
 6   Expiration Date     294 non-null    object 
 7   Strike Price        294 non-null    float64
 8   Time to Expiration  294 non-null    float64
dtypes: float64(6), object(3)
memory usage: 23.0+ KB


In [6]:
data.to_csv('data/data.csv', index=False)